# GenAI Agentic Automation with AutoGen

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

## Let's dive into the example code
The relevant code is available below. You may dive deeper to see how AutoGen easy to use and how powerful it is.

Note: GenAI may produce different results each time it runs. However, by default, AutoGen uses a cache to generate results, so the output will be the same as the first time you run the code. To generate new results, you can delete the .cache folder. Sometimes, the code_writer_agent and the code_executor_agent may not be able to resolve their bugs. If this happens, you can delete the cache and try again.

### Setup

In [1]:
# Reading the .env file for the OpenAI API key
# This example uses OpenAI as the LLM. You will need OPENAI_API_KEY=<your key> in your .env file
import os
env_vars = !cat .env
for var in env_vars:
    key, value = var.split('=')
    os.environ[key] = value

In [2]:
llm_config = {"model": "gpt-3.5-turbo"}

### Create agents 

In [3]:
from autogen import ConversableAgent, AssistantAgent

In [4]:
code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=llm_config,
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "coding",
        "use_docker": False,
    },
)

code_writer_agent = AssistantAgent(
    name="code_writer_agent",
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "It is good to go" in msg["content"] or "Goodbye" in msg["content"] or "TERMINATE" in msg['content'],
)

service_manager = AssistantAgent(
    name="service_manager",
    llm_config=llm_config,
    system_message="You are a service manager who creats new service to help customer implement the solution. "
        "When the service is ready, say 'It is good to go'. "
        "Begin the service creation by stating your role. "
        "Think about the business values the solution can provide and how to use the solution to innovate. ",
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "It is good to go" in msg["content"] or "Goodbye" in msg["content"] or "TERMINATE" in msg['content'],
)

prop_manager = AssistantAgent(
    name="prop_manager",
    llm_config=llm_config,
    system_message="You are a propositon manager who creats new proposition based on the new service. "
    "First, You need to create a name for this proposition. "
    "You need to briefly describe this proposition."
    "You need to create a problem statement. "
    "Provide the target audience of the proposition. "
    "You need to list out the features the proposition provide. "
    "List out benefits to customer of the proposition. "
    "List out the USPs of our company. You may these USPs of our company as reference: \n"
    "Expertise and track record in securely integrating disruptive technologies into highly regulated organisations \n"
    "Tailored approach to address specific adoption challenges in security for your specific needs \n"
    "Multidisciplinary approach, drawing on expertise in platform business functions, engaging stakeholders, central governance teams, and technical experts \n"
    "Systematic approach aligned to the latest strategy and technology innovations \n\n"
    "When the proposition is ready, say 'It is good to go'. "
    "Begin the proposition creation by stating your role. ",
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "Goodbye" in msg["content"] or "TERMINATE" in msg['content'],
)

### Solution source

In [5]:
prop_info_source = "https://airwalkreply.com/finops-approach-in-azure"

### Initial chat message

In [6]:
message = f"Download {prop_info_source} by using curl -o. Use Python with BeautifulSoap to read the content of the downloaded html file and ignore all markup."

### Sequential chats

In [7]:
chats = [
    {   
        "sender": code_executor_agent,
        "recipient": code_writer_agent,
        "message": message,
        "summary_method": "reflection_with_llm",
        "summary_args": {"summary_prompt" : 
            "The downloaded content is the information of the new solution. Summarise and save the information as the soluiton to JSON object: {'Solution': ''}"
        },
    },
    {
        "sender": code_writer_agent,
        "recipient": service_manager,
        "summary_method": "reflection_with_llm",
        "summary_args": {"summary_prompt" : 
            "Save the service and its details created by service_manager to JSON object: {'Service': '', 'Service Details': '',}"
        },
    },
    {
        "sender": service_manager,
        "recipient": prop_manager,
        "summary_method": "last_msg",
    },
]

### Start the chats

In [8]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
code_executor_agent (to code_writer_agent):

Download https://airwalkreply.com/finops-approach-in-azure by using curl -o. Use Python with BeautifulSoap to read the content of the downloaded html file and ignore all markup.

--------------------------------------------------------------------------------
code_writer_agent (to code_executor_agent):

```sh
# filename: download_page.sh
curl -o finops-approach-in-azure.html https://airwalkreply.com/finops-approach-in-azure
```

```python
# filename: read_html_content.py
from bs4 import BeautifulSoup

with open('finops-approach-in-azure.html', 'r') as file:
    soup = BeautifulSoup(file, 'html.parser')
    text = soup.get_text()

print(text)
```

Please execute the shell script first to download the webpage. Then, run the Python script to read the content o

/opt/homebrew/lib/python3.11/site-packages/autogen/agentchat/chat.py:114: UserWarning: message is not provided in a chat_queue entry. input() will be called to get the initial message.
  warnings.warn(


code_writer_agent (to service_manager):


Context: 
```json
{
    "Solution": "The FinOps approach is gaining popularity in cloud computing, particularly in Microsoft Azure. FinOps is a portmanteau of finance and DevOps, combining financial management, cloud governance, and cloud cost optimization. Azure provides features like Cost Management, Cost alerts, Budgets, and Advisor recommendations for monitoring, analyzing, and optimizing cloud costs. By using Azure Cost Analysis and tagging resources accurately, organizations can track and manage their cloud spending effectively. Budget alerts and proactive spending limits help in controlling costs. Azure Advisor offers recommendations to optimize Azure resources based on best practices and cost-saving opportunities. Overall, the FinOps approach in Azure enables organizations to manage and optimize their cloud costs while meeting technical and business objectives."
}
```

--------------------------------------------------------------------

## Chat summary

In [9]:
i=0
for chat_result in chat_results:
    i+=1
    print(f"Summary of Chat{i}")
    print(chat_result.summary)
    print("\n")


Summary of Chat1
```json
{
    "Solution": "The FinOps approach is gaining popularity in cloud computing, particularly in Microsoft Azure. FinOps is a portmanteau of finance and DevOps, combining financial management, cloud governance, and cloud cost optimization. Azure provides features like Cost Management, Cost alerts, Budgets, and Advisor recommendations for monitoring, analyzing, and optimizing cloud costs. By using Azure Cost Analysis and tagging resources accurately, organizations can track and manage their cloud spending effectively. Budget alerts and proactive spending limits help in controlling costs. Azure Advisor offers recommendations to optimize Azure resources based on best practices and cost-saving opportunities. Overall, the FinOps approach in Azure enables organizations to manage and optimize their cloud costs while meeting technical and business objectives."
}
```


Summary of Chat2
{
    "Service": "FinOps Optimization Service in Microsoft Azure",
    "Service Detai

## Costs for running the agents

In [10]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.00294, 'gpt-3.5-turbo-0125': {'cost': 0.00294, 'prompt_tokens': 5013, 'completion_tokens': 289, 'total_tokens': 5302}}, 'usage_excluding_cached_inference': {'total_cost': 0.00294, 'gpt-3.5-turbo-0125': {'cost': 0.00294, 'prompt_tokens': 5013, 'completion_tokens': 289, 'total_tokens': 5302}}}


{'usage_including_cached_inference': {'total_cost': 0.0042715, 'gpt-3.5-turbo-0125': {'cost': 0.0042715, 'prompt_tokens': 5831, 'completion_tokens': 904, 'total_tokens': 6735}}, 'usage_excluding_cached_inference': {'total_cost': 0.0042715, 'gpt-3.5-turbo-0125': {'cost': 0.0042715, 'prompt_tokens': 5831, 'completion_tokens': 904, 'total_tokens': 6735}}}


{'usage_including_cached_inference': {'total_cost': 0.0022770000000000004, 'gpt-3.5-turbo-0125': {'cost': 0.0022770000000000004, 'prompt_tokens': 1386, 'completion_tokens': 1056, 'total_tokens': 2442}}, 'usage_excluding_cached_inference': {'total_cost': 0.0022770000000000004, 'gpt-3.5-turbo-01